---
# Asymmetric FOD estimation tutorial

**Matteo Bastiani, FMRIB**

May 2018

---

This jupyter notebook explaines how to estimate asymmetric FODs from dMRI data using the qboot_v2 python package.
Both FSL and the qboot_v2 python package need to have been already downloaded and succesfully installed.

In this tutorial, we will use dMRI data provided by the FSL course.
Data can be downloaded from the following link:
https://fsl.fmrib.ox.ac.uk/fslcourse/downloads/fdt.tar.gz

Once the download is complete, unpack the fdt.tar.gz file and change the following fsl_data_path and tutorial path accordingly.

In [ ]:
fsl_data_path = '/Users/matteob/Downloads/fdt/fdt2/subj1'
tutorial_path = '/Users/matteob/qboot_v2/aFOD/tutorial'

In [ ]:
%%bash -s "{fsl_data_path}"

mkdir -p ${1}/dti
${FSLDIR}/bin/dtifit -k ${1}/data.nii.gz -o ${1}/dti/dti -m ${1}/nodif_brain_mask -r ${1}/bvecs -b ${1}/bvals  --kurt

${FSLDIR}/bin/fslmaths ${1}/dti/dti_FA.nii.gz -thr 0.4 -ero -bin ${1}/dti/high_fa_mask

In [ ]:
# Import the csdeconv module
from aFOD.csdeconv import csdeconv as csd

## White matter response function estimation

In [ ]:
# Estimate the white matter response function from high FA voxels up to spherical harmoic order of 4
resp_max_harmonic_order = 4

r_wm = csd.Response.get_response(fsl_data_path + '/data.nii.gz',
                              fsl_data_path + '/dti/high_fa_mask.nii.gz',
                              fsl_data_path + '/bvals',
                              fsl_data_path + '/bvecs',
                              resp_max_harmonic_order,
                              dti_basename = fsl_data_path + '/dti/dti')

The response function estimation method has correctly identified the 4 different b-shells in the dataset (0, 500, 1500, 2500 s/mm^2). It has estimated one response function per b-shell and stored its spherical harmonics coefficients in a [N_shells X N_even_coefficients] array.

We can save the coefficients to a text file for reference and future use:

In [ ]:
# Store the response coefficients to a text file
r_wm.write_coefficients(fsl_data_path + '/response_wm_l4.txt')

## Estimate aFODs

Now, we can estimate the aFODS using the estimated response function.

In [ ]:
# To speed things up, let's mask only 3 coronal slices...
import nibabel as nib

mask_obj = nib.load(fsl_data_path + '/nodif_brain_mask.nii.gz')
mask = mask_obj.get_data()

mask[:, 0:56, :] = 0
mask[:, 59:, :] = 0

nib.Nifti1Image(mask, None, mask_obj.header).to_filename(fsl_data_path + '/nodif_brain_mask_roi.nii.gz')

In [ ]:
# Estimate FODs up to harmonic order 8
max_fod_harmonic_order = 8

In [ ]:
afod = csd.csdeconv(r_wm,
                    fsl_data_path + '/data.nii.gz',
                    fsl_data_path + '/nodif_brain_mask_roi.nii.gz',
                    fsl_data_path + '/bvals', 
                    fsl_data_path + '/bvecs',
                    max_fod_harmonic_order,
                    sym = False,
                    out_file = fsl_data_path + '/MS_afod.nii.gz')

For comparison, we can also estimate the symmetric FODs using multi-shell CSD.

In [ ]:
sfod = csd.csdeconv(r_wm,
                    fsl_data_path + '/data.nii.gz',
                    fsl_data_path + '/nodif_brain_mask_roi.nii.gz',
                    fsl_data_path + '/bvals', 
                    fsl_data_path + '/bvecs',
                    max_fod_harmonic_order,
                    sym = True,
                    out_file = fsl_data_path + '/MS_sfod.nii.gz')

Once both asymmetric and symmetric FODs fit have run, it is possible to visualise the results using FSLeyes. Once you have looked at the results, close FSLeyes and carry on.

In [ ]:
%%bash -s "{fsl_data_path}"

fsleyes ${1}/dti/dti_FA.nii.gz -dr 0 0.5 -in spline \
        ${1}/MS_afod.nii.gz -ot sh -sr 10 \
        ${1}/MS_sfod.nii.gz -ot sh -sr 10 &


## Peaks extraction

To extract peaks from the FODs, run the following commands:

In [ ]:
import os.path as op
import numpy as np
from aFOD.utils.utils import get_peaks

In [ ]:
# Read the sphere text file; this contains:
# [x1 y1 z1 n11 n12 n13 -1]
# [x2 y2 z2 n21 n22 n23 n24]
# [x3 y3 z3 n31 n32 n33 n34]
# [...]
# [xN yN zN nN1 nN2 nN3 nN4]
# Where x, y and z are the vertex coordinates and n are the neighbouring vertices index (-1 if none).
resource_dir = op.dirname(csd.__file__)
ico5 = op.join(resource_dir, 'ico_5.txt')

sphere = np.genfromtxt(ico5, dtype=np.float)

In [ ]:
# Extract up to 6 peaks from aFODs using non-linear optimisation.
# Note that we need to extract so many peaks beacuse of the FOD being asymmetric.
# In the case of symmetric FODs, 3 would be enough
n_peaks = 6

peaks, amplitudes = get_peaks(sphere,
                              fsl_data_path + '/MS_afod.nii.gz',
                              fsl_data_path + '/nodif_brain_mask_roi.nii.gz',
                              max_fod_harmonic_order, 
                              sym=False, 
                              save_results=True, 
                              n=n_peaks, 
                              non_lin=True)

Peaks extracted from aFODs can be visualised in FSLeyes, specifying that they should be interpreted as directed. Once you have looked at the results, close FSLeyes and carry on.

In [ ]:
%%bash -s "{fsl_data_path}"

fsleyes ${1}/dti/dti_FA.nii.gz -dr 0 0.5 -in spline \
        ${1}/MS_afod.nii.gz -ot sh -sr 10 \
        ${1}/peak1.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 \
        ${1}/peak2.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 \
        ${1}/peak3.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 \
        ${1}/peak4.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 \
        ${1}/peak5.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 \
        ${1}/peak6.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 &


## Multi-tissue aFODs

It is possible to specify multiple response functions, one for each tissu-type.
Response function can be estimated as before, i.e., providing a binary mask obtained from, e.g., a random seleciton of  voxels from a gray matter or csf segmentation.

For the purpose of this tutorial, we have included the estimated 0-th order spherical harmonics coefficients for gray matter and csf in the tutorial folder. We can load them using the following commands:

In [ ]:
r_gm = csd.Response.read_coefficients(tutorial_path + '/response_gm_l0.txt')

r_csf = csd.Response.read_coefficients(tutorial_path + '/response_csf_l0.txt')

Now we can run the aFOD estimation commands including the different response functions.

**NOTE**
The function assumes that the first reponse function in the list is always be the white matter one!!!

In [ ]:
# We need to tell the function the harmonic orders of the different tissue-specific FODs
max_fod_harmonic_order_mt = [8, 0, 0]

afod = csd.csdeconv([r_wm, r_gm, r_csf],
                    fsl_data_path + '/data.nii.gz',
                    fsl_data_path + '/nodif_brain_mask_roi.nii.gz',
                    fsl_data_path + '/bvals', 
                    fsl_data_path + '/bvecs',
                    max_fod_harmonic_order_mt,
                    sym = False,
                    out_file = fsl_data_path + '/MSMT_afod.nii.gz')

To visualise the white matter fods, we need to select a subset of the results, i.e., the first 81 SH coefficients.

In [ ]:
%%bash -s "{fsl_data_path}"

${FSLDIR}/bin/fslroi ${1}/MSMT_afod.nii.gz ${1}/MSMT_afod_wm.nii.gz 0 81

fsleyes ${1}/dti/dti_FA.nii.gz -dr 0 0.5 -in spline \
        ${1}/MSMT_afod_wm.nii.gz -ot sh -sr 10

